In [0]:
import tensorflow as tf

growthFactor = k = 32

def UpSamplingNearest(tensor, target_height, target_width):
  '''
    Will be used for Nearest Neighbor Interpolation
  '''
  return tf.image.resize(tensor, size=[target_height, target_width], method='nearest', name='UpSamplingNearest')

def BottleneckLayer(tensor, name_string):
  '''
    Made to add the BN ReLU Conv style bottleneck
  '''
  name_BN = name_string + '_BN'
  name_ReLU = name_string + '_ReLU'
  tensor = tf.keras.layers.BatchNormalization(name=name_BN)(tensor)
  tensor = tf.keras.layers.ReLU(name=name_ReLU)(tensor)
  return tensor

def DenseBlock(tensor, iterations=1, namestring=''):
  '''
    This is for increasing the channel size of feature map.

    @param: 'iterations' will be used as the gain growthFactor per iterations
  '''
  for i in range(iterations):
    out = tensor
    tensor = BottleneckLayer(tensor, name_string=namestring + '_' + str(i+1) + '_BottleNeck1')
    tensor = tf.keras.layers.SeparableConv2D(filters=k, kernel_size=3, dilation_rate=3, padding='same', name=namestring + '_' + str(i+1) + '_Conv1')(tensor)
    tensor = BottleneckLayer(tensor, name_string=namestring + '_' + str(i+1) + '_BottleNeck2')
    tensor = tf.keras.layers.SeparableConv2D(filters=k // 2, kernel_size=3, dilation_rate=2, padding='same', name=namestring + '_' + str(i+1) + '_Conv2')(tensor)
    tensor = BottleneckLayer(tensor, name_string=namestring + '_' + str(i+1) + '_BottleNeck3')
    tensor = tf.keras.layers.SeparableConv2D(filters=k // 4, kernel_size=3, padding='same', name=namestring + '_' + str(i+1) + '_Conv3', activation='relu')(tensor)

    tensor_shape = tf.keras.backend.int_shape(tensor)
    height = tensor_shape[1]
    width = tensor_shape[2]

    # -- Pyramid Network -- 
    branch32 = tf.keras.layers.AveragePooling2D(pool_size=(32,32), name=namestring + '_' + str(i+1) + '_Pyramid_AvgPool32')(tensor) # 1/32 times the feature map
    branch16 = tf.keras.layers.AveragePooling2D(pool_size=(16,16), name=namestring + '_' + str(i+1) + '_Pyramid_AvgPool16')(tensor) # 1/16 times the feature map
    branch8 = tf.keras.layers.AveragePooling2D(pool_size=(8,8), name=namestring + '_' + str(i+1) + '_Pyramid_AvgPool8')(tensor)     # 1/8  times the feature map
    branch4 = tf.keras.layers.AveragePooling2D(pool_size=(4,4), name=namestring + '_' + str(i+1) + '_Pyramid_AvgPool4')(tensor)     # 1/4 times the feature map

    branch32 = tf.keras.layers.SeparableConv2D(filters=k // 4, kernel_size=3, padding='same', dilation_rate=2, name=namestring + '_' + str(i+1) + '_Pyramid_Conv32', activation='relu')(branch32)
    branch16 = tf.keras.layers.SeparableConv2D(filters=k // 4, kernel_size=3, padding='same', dilation_rate=2, name=namestring + '_' + str(i+1) + '_Pyramid_Conv16', activation='relu')(branch16)
    branch8 = tf.keras.layers.SeparableConv2D(filters=k // 4, kernel_size=3, padding='same', dilation_rate=2, name=namestring + '_' + str(i+1) + '_Pyramid_Conv8', activation='relu')(branch8)
    branch4 = tf.keras.layers.SeparableConv2D(filters=k // 4, kernel_size=3, padding='same', dilation_rate=2, name=namestring + '_' + str(i+1) + '_Pyramid_Conv4', activation='relu')(branch4)

    branch32 = tf.keras.layers.Lambda(UpSamplingNearest, arguments={'target_height':height, 'target_width':width}, name=namestring + '_' + str(i+1) + '_Pyramid_Upsampling32')(branch32)
    branch16 = tf.keras.layers.Lambda(UpSamplingNearest, arguments={'target_height':height, 'target_width':width}, name=namestring + '_' + str(i+1) + '_Pyramid_Upsampling16')(branch16)
    branch8 = tf.keras.layers.Lambda(UpSamplingNearest, arguments={'target_height':height, 'target_width':width}, name=namestring + '_' + str(i+1) + '_Pyramid_Upsampling8')(branch8)
    branch4 = tf.keras.layers.Lambda(UpSamplingNearest, arguments={'target_height':height, 'target_width':width}, name=namestring + '_' + str(i+1) + '_Pyramid_Upsampling4')(branch4)

    tensor = tf.keras.layers.Concatenate(name=namestring + '_' + str(i+1) + '_Pyramid_Concat1')([branch32, branch16, branch8, branch4])
    tensor = BottleneckLayer(tensor, name_string=namestring + '_' + str(i+1) + '_Pyramid_BottleNeck')
    tensor = tf.keras.layers.SeparableConv2D(filters=k, kernel_size=3, padding='same', name=namestring + '_' + str(i+1) + '_Pyramid_Conv')(tensor)
    tensor = tf.keras.layers.Concatenate(name=namestring + '_' + str(i+1) + '_Pyramid_Concat2')([tensor, out])
    tensor = BottleneckLayer(tensor, name_string=namestring + '_' + str(i+1) + '_BottleNeck4')
    tensor_shape = tf.keras.backend.int_shape(tensor)
    channels = tensor_shape[-1]
    tensor = tf.keras.layers.SeparableConv2D(filters=channels, kernel_size=3, padding='same', dilation_rate=2, name=namestring + '_' + str(i+1) + '_Conv4', activation='relu')(tensor)
  return tensor

def TransitionBlock(tensor, cardinality=1, isUpward=False, namestring=''):
  '''
    This is for directly manipulating the feature map size.

    @param: 'cardinality' is the number of paths to be used.
        For Down: chosen 16
        For Up: chosen 8
  '''
  iterations = cardinality
  tensor_shape = tf.keras.backend.int_shape(tensor)
  channels = tensor_shape[-1]
  small_channel = (channels) // iterations
  if isUpward: # No gain in channel size
    height = tensor_shape[1] 
    width = tensor_shape[2]

    target_height = 2 * height
    target_width = 2 * width

    tensor = tf.keras.layers.Lambda(UpSamplingNearest, arguments={'target_height':target_height, 'target_width':target_width}, name=namestring + '_Up_Upsampling')(tensor)
    tensor = tf.keras.layers.Conv2D(filters=channels, kernel_size=3, padding='same', dilation_rate=3, name=namestring + '_Up_MixingConv1')(tensor)
    
    tensor = BottleneckLayer(tensor, name_string=namestring + '_Up_BottleNeck1')
    tensor = tf.keras.layers.SeparableConv2D(filters=channels, kernel_size=3, padding='same', dilation_rate=3, name=namestring + '_Up_Conv1')(tensor)

    group = []
    _input = tensor
    for i in range(iterations):
      tensor = BottleneckLayer(_input, name_string=namestring + '_' + str(i+1) + '_Up_Paths_BottleNeck1')
      tensor = tf.keras.layers.SeparableConv2D(filters=small_channel, kernel_size=3, dilation_rate=3, padding='same', name=namestring + '_' + str(i+1) + '_Up_Paths_Conv1')(tensor)
      tensor = BottleneckLayer(tensor, name_string=namestring + '_' + str(i+1) + '_Up_Paths_BottleNeck2')
      tensor = tf.keras.layers.SeparableConv2D(filters=small_channel, kernel_size=3, padding='same', name=namestring + '_' + str(i+1) + '_Up_Paths_Conv2')(tensor)
      group.append(tensor)
    
    tensor = tf.keras.layers.Concatenate(name=namestring + '_Up_Paths_Concat')(group)
    tensor = BottleneckLayer(tensor, name_string=namestring + '_Up_BottleNeck2')
    tensor = tf.keras.layers.Conv2D(filters=channels, kernel_size=1, name=namestring + '_Up_MixingConv2')(tensor)

    _input = tf.keras.layers.Conv2D(filters=channels, kernel_size=1, name=namestring + '_Up_ChannelSize_Dec')(_input)
    tensor = tf.keras.layers.Concatenate(name=namestring + '_Up_ResNeXt_Concat')([tensor, _input])

    tensor = BottleneckLayer(tensor, name_string=namestring + '_Up_BottleNeck3')
    tensor = tf.keras.layers.SeparableConv2D(filters=channels, kernel_size=3, dilation_rate=3, padding='same', name=namestring + '_Up_MixingConv3', activation='relu')(tensor)

    return tensor

  else: #NO GAIN In channel Number
    group = []
    _input = tensor

    for i in range(iterations):
      tensor = BottleneckLayer(_input, name_string=namestring + '_' + str(i+1) + '_Down_Paths_BottleNeck1')
      tensor = tf.keras.layers.SeparableConv2D(filters=small_channel, kernel_size=3, padding='same', name=namestring + '_' + str(i+1) + '_Down_Paths_Conv1', dilation_rate=3)(tensor)
      tensor = BottleneckLayer(tensor, name_string=namestring + '_' + str(i+1) + '_Down_Paths_BottleNeck2')
      tensor = tf.keras.layers.SeparableConv2D(filters=small_channel, kernel_size=3, padding='same', strides=2, name=namestring + '_' + str(i+1) + '_Down_Paths_Conv2')(tensor)
      group.append(tensor)

    tensor = tf.keras.layers.Concatenate(name=namestring + '_Down_Paths_Concat')(group)

    tensor = BottleneckLayer(tensor, name_string=namestring + '_Down_BottleNeck1')
    tensor = tf.keras.layers.SeparableConv2D(filters=channels, kernel_size=3, padding='same', dilation_rate=3, name=namestring + '_Down_Conv1')(tensor)
    tensor = BottleneckLayer(tensor, name_string=namestring + '_Down_BottleNeck2')
    tensor = tf.keras.layers.SeparableConv2D(filters=channels, kernel_size=3, padding='same', dilation_rate=3, name=namestring + '_Down_Conv2')(tensor)

    _input = BottleneckLayer(_input, name_string=namestring + '_Down_BottleNeck3')
    _input = tf.keras.layers.SeparableConv2D(filters=channels, kernel_size=3, padding='same', strides=2, name=namestring + '_Down_Conv3')(_input)
    _input = BottleneckLayer(_input, name_string=namestring + '_Down_BottleNeck4')
    _input = tf.keras.layers.SeparableConv2D(filters=channels, kernel_size=3, padding='same', dilation_rate=3, name=namestring + '_Down_Conv4')(_input)

    tensor = tf.keras.layers.Concatenate(name=namestring + '_Down_ResNeXt_Concat')([_input, tensor])
    tensor = tf.keras.layers.SeparableConv2D(filters=channels, kernel_size=3, padding='same', dilation_rate=3, name=namestring + '_Down_MixingConv5', activation='relu')(tensor)
    return tensor

def WeepingAngel(input_tensor, input_shape=(256,256,3), _classes=21):
  x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', name='IntroConv1', dilation_rate=3)(input_tensor)
  x = BottleneckLayer(x, name_string='Main_Net_BottleNeck1')
  conv256 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='IntroConv2', dilation_rate=3)(x)

  # Inc. channels from (256,256,64) to (256,256,128) 2 iterations
  x = DenseBlock(tensor=conv256, iterations=2, namestring='DenseBlock1')

  # Dropping the feature map size to (128,128,128) with cardinality 32
  conv128 = TransitionBlock(tensor=x, cardinality=16, isUpward=False, namestring='TransitionBlock1')

  # Inc. channels from (128,128,128) to (128,128,256) 4 iterations
  x = DenseBlock(tensor=conv128, iterations=4, namestring='DenseBlock2')

  # Dropping the feature map size to (64,64,256) with cardinality 32
  conv64 = TransitionBlock(tensor=x, cardinality=16, isUpward=False, namestring='TransitionBlock2')

  # Inc. channels from (64,64,256) to (64,64,512) 8 iterations
  x = DenseBlock(tensor=conv64, iterations=8, namestring='DenseBlock3')

  # Dropping the feature map size to (32,32,512) with cardinality 32
  conv32 = TransitionBlock(tensor=x, cardinality=16, isUpward=False, namestring='TransitionBlock3')

  # -- PYRAMID Pooling --
  
  x1 = tf.keras.layers.AveragePooling2D(pool_size=(32,32), name='AvgPool2D_Feature_Dec_32')(conv32) # x1 tells the final size
  x1 = tf.keras.layers.Conv2D(filters=64, kernel_size=1, name='SepConv_adjustFilters', activation='relu')(x1)
  x2 = tf.keras.layers.SeparableConv2D(filters=64, kernel_size=3, padding='valid', dilation_rate=15, name='Feature_map_Dec_16', activation='relu')(conv32)
  x8 = tf.keras.layers.SeparableConv2D(filters=64, kernel_size=3, padding='valid', dilation_rate=12, name='Feature_map_Dec_8', activation='relu')(conv32)
  x16 = tf.keras.layers.SeparableConv2D(filters=64, kernel_size=3, padding='valid', dilation_rate=8, name='Feature_map_Dec_4', activation='relu')(conv32)

    # upsampling
  x1 = tf.keras.layers.Lambda(UpSamplingNearest, arguments={'target_height':32, 'target_width':32}, name='Pyramid_Upsampling_32')(x1)
  x2 = tf.keras.layers.Lambda(UpSamplingNearest, arguments={'target_height':32, 'target_width':32}, name='Pyramid_Upsampling_16')(x2)
  x8 = tf.keras.layers.Lambda(UpSamplingNearest, arguments={'target_height':32, 'target_width':32}, name='Pyramid_Upsampling_8')(x8)
  x16 = tf.keras.layers.Lambda(UpSamplingNearest, arguments={'target_height':32, 'target_width':32}, name='Pyramid_Upsampling_4')(x16)

  x1 = tf.keras.layers.Conv2D(filters=64, kernel_size=1, name='MixingConv_32', activation='relu')(x1)
  x2 = tf.keras.layers.Conv2D(filters=64, kernel_size=1, name='MixingConv_16', activation='relu')(x2)
  x8 = tf.keras.layers.Conv2D(filters=64, kernel_size=1, name='MixingConv_8', activation='relu')(x8)
  x16 = tf.keras.layers.Conv2D(filters=64, kernel_size=1, name='MixingConv_4', activation='relu')(x16)

  x = tf.keras.layers.Concatenate()([x1,x2,x8,x16])
  conv32 = tf.keras.layers.Conv2D(filters=256, kernel_size=1, name='PointConv_adjust_conv32', activation='relu')(conv32)
  x = tf.keras.layers.Concatenate()([x, conv32])
  x = tf.keras.layers.Conv2D(filters=512, kernel_size=1, name='MixingConv1', activation='relu')(x)

  # -- END OF Pyramid Pooling -- We have(32,32,512)
  x = BottleneckLayer(x, name_string='Main_Net_BottleNeck2')
  # we will be doing 1/4 of channel for dense block to act: Here we have (32,32,512)
  x = tf.keras.layers.SeparableConv2D(filters=128, kernel_size=3, padding='same', dilation_rate=3, name='Conv_ChannelReduction_to128')(x)
  x = BottleneckLayer(x, name_string='Main_Net_BottleNeck3')
  x = tf.keras.layers.Conv2D(filters=128, kernel_size=1, name='MixingConv2', activation='relu')(x) #It's a (32,32,128)

  # Transition Block from (32,32,128) to (64,64,128)
  x = TransitionBlock(x, cardinality=8, isUpward=True, namestring='TransitionBlock4')
  conv64 = tf.keras.layers.Conv2D(filters=128, kernel_size=1, activation='relu', name='Conv64_ChannelSize_Dec')(conv64)
  x = tf.keras.layers.Add(name='Main_Net_Add64')([x,conv64])

  # Dense Block from (64,64,128) to get (64,64,256)
  x = DenseBlock(x, iterations=4, namestring='DenseBlock4') # Result: (64,64,256)

  x = BottleneckLayer(x, name_string='Main_Net_BottleNeck4')
  x = tf.keras.layers.SeparableConv2D(filters=64, kernel_size=3, padding='same', dilation_rate=3, name='Conv_ChannelReduction_to64')(x)
  x = BottleneckLayer(x, name_string='Main_Net_BottleNeck5')
  x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, name='MixingConv3', activation='relu')(x) # It's (64,64,64)

  # Transition Block from (64,64,64) to (128,128,64)
  x = TransitionBlock(x, cardinality=8, isUpward=True, namestring='TransitionBlock5')
  conv128 = tf.keras.layers.Conv2D(filters=64, kernel_size=1, activation='relu', name='Conv128_ChannelSize_Dec')(conv128)
  x = tf.keras.layers.Add(name='Main_Net_Add128')([x,conv128])

  # Dense Block from (128,128,64) to (128,128,128)
  x = DenseBlock(x, iterations=2, namestring='DenseBlock5') # Result: (128,128,128)

  x = BottleneckLayer(x, name_string='Main_Net_BottleNeck6')
  x = tf.keras.layers.SeparableConv2D(filters=32, kernel_size=3, padding='same', dilation_rate=3, name='Conv_ChannelReduction_to32')(x)
  x = BottleneckLayer(x, name_string='Main_Net_BottleNeck7')
  x = tf.keras.layers.Conv2D(filters=32, kernel_size=1, name='MixingConv4', activation='relu')(x) # It's (128,128,32)

  # Transition Block from (128,128,32) to (256,256,32)
  x = TransitionBlock(x, cardinality=8, isUpward=True, namestring='TransitionBlock6')
  conv256 = tf.keras.layers.Conv2D(filters=32, kernel_size=1, activation='relu', name='Conv256_ChannelSize_Dec')(conv256)
  x = tf.keras.layers.Add(name='Main_Net_Add256')([x,conv256])

  # Dense Block from (256, 256, 32) to (256, 256, 64)
  x = DenseBlock(x, iterations=1, namestring='DenseBlock6') # Result:(256,256,64)

  x = tf.keras.layers.SeparableConv2D(filters=64, kernel_size=3, dilation_rate=3, padding='same', name='Final_Refinement')(x)

  x = tf.keras.layers.Conv2D(_classes, kernel_size=1, name='Pre_Softmax_Conv')(x)
  x = tf.keras.layers.Activation('softmax')(x)

  return tf.keras.Model(inputs=input_tensor, outputs=x, name='Weeping Angel')

In [0]:
Input = tf.keras.Input(shape=(256,256,3))
model = WeepingAngel(Input, input_shape=(256,256,3), _classes=21)

In [0]:
model.summary()

Model: "Weeping Angel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
IntroConv1 (Conv2D)             (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
Main_Net_BottleNeck1_BN (BatchN (None, 256, 256, 64) 256         IntroConv1[0][0]                 
__________________________________________________________________________________________________
Main_Net_BottleNeck1_ReLU (ReLU (None, 256, 256, 64) 0           Main_Net_BottleNeck1_BN[0][0]    
______________________________________________________________________________________